In [1]:
from datasets import load_dataset

dataset = load_dataset("Xenova/quickdraw-small")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4500000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/250000 [00:00<?, ? examples/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4500000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 250000
    })
    valid: Dataset({
        features: ['image', 'label'],
        num_rows: 250000
    })
})

In [5]:
train_data = dataset['train']
tf_ds = train_data.to_tf_dataset(
            columns=["image"],
            label_cols=["label"],
            batch_size=256,
            shuffle=True
            )

C:\ProgramData\anaconda3\envs\py39\lib\site-packages\datasets\arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [21]:
from tensorflow.keras import layers, Model

def create_custom_vgg_28(classes=345):
    input_shape = (28, 28, 1)  # Assuming RGB images for this example

    inputs = layers.Input(shape=input_shape)
    
    # Custom VGG-like architecture for 28x28 input size
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the custom VGG-like model with 28x28 input size and 340 output classes
model = create_custom_vgg_28(classes=345)

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Display model summary
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 128)       147584    
                                                           

In [28]:
model = tf.keras.models.load_model('quickdraw.h5', compile=False)

# Compile the model with your preferred optimizer and its settings
opt = tf.keras.optimizers.Adam(learning_rate=0.001)  # You can replace this with your optimizer and settings
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
import numpy as np

In [34]:
np.argmax(model.predict(np.array(train_data[0]['image']).reshape(1,28,28,1)))

1/1 [==============================] - 0s 59ms/step


260

In [35]:
train_data[0]['label']

260

In [45]:

def predict(img):
    try:
        img = np.array(img).reshape(1, 28, 28,1)
        out = model.predict(img);
        top5 = np.argsort(out[0])[::-1][:5]
        labels = train_data.features['label'].names

        # Create a dictionary with label-probability pairs for the top 5 predictions
        probabilities = {labels[i]: float(out[0][i]) for i in top5}

        return probabilities
    except:
        return {"None":1.0}


In [46]:
labels = train_data.features['label'].names
for i in range(10):
    print(predict(train_data[i]['image']),labels[train_data[i]['label']])

1/1 [==============================] - 0s 99ms/step
{'shorts': 0.6542353630065918, 'underwear': 0.3270261585712433, 'pants': 0.007486306130886078, 'keyboard': 0.0016458355821669102, 'piano': 0.0011699199676513672} shorts
1/1 [==============================] - 0s 41ms/step
{'octopus': 0.9999284744262695, 'mosquito': 1.8197912140749395e-05, 'spider': 1.097452150133904e-05, 'beard': 8.554997293686029e-06, 'necklace': 8.110391718219034e-06} octopus
1/1 [==============================] - 0s 47ms/step
{'trumpet': 0.8191449046134949, 'trombone': 0.18055731058120728, 'clarinet': 8.8935557869263e-05, 'whale': 2.2057834939914756e-05, 'aircraft carrier': 2.19633129745489e-05} trumpet
1/1 [==============================] - 0s 51ms/step
{'parachute': 0.26201072335243225, 'table': 0.20394177734851837, 'dog': 0.1001933291554451, 'ant': 0.04689065366983414, 'piano': 0.03941114991903305} frog
1/1 [==============================] - 0s 46ms/step
{'camouflage': 0.12378180772066116, 'blackberry': 0.0336980

In [52]:
!pip install -q gradio==3.50.2

In [59]:
import gradio as gr

In [60]:
gr.Sketchpad

AttributeError: module 'gradio' has no attribute 'Sketchpad'

In [61]:
gr.Interface(fn=predict,
             inputs='sketchpad',
             outputs="label",
             live=True).launch(debug=True)

ValueError: No such component or layout: sketchpad

In [63]:
demo = gr.Interface(predict, 
           inputs=gr.Sketchpad(invert_colors=False),
                     outputs='image')

AttributeError: module 'gradio' has no attribute 'Sketchpad'

In [64]:
train_data.features['label'].names

['aircraft carrier',
 'airplane',
 'alarm clock',
 'ambulance',
 'angel',
 'animal migration',
 'ant',
 'anvil',
 'apple',
 'arm',
 'asparagus',
 'axe',
 'backpack',
 'banana',
 'bandage',
 'barn',
 'baseball bat',
 'baseball',
 'basket',
 'basketball',
 'bat',
 'bathtub',
 'beach',
 'bear',
 'beard',
 'bed',
 'bee',
 'belt',
 'bench',
 'bicycle',
 'binoculars',
 'bird',
 'birthday cake',
 'blackberry',
 'blueberry',
 'book',
 'boomerang',
 'bottlecap',
 'bowtie',
 'bracelet',
 'brain',
 'bread',
 'bridge',
 'broccoli',
 'broom',
 'bucket',
 'bulldozer',
 'bus',
 'bush',
 'butterfly',
 'cactus',
 'cake',
 'calculator',
 'calendar',
 'camel',
 'camera',
 'camouflage',
 'campfire',
 'candle',
 'cannon',
 'canoe',
 'car',
 'carrot',
 'castle',
 'cat',
 'ceiling fan',
 'cell phone',
 'cello',
 'chair',
 'chandelier',
 'church',
 'circle',
 'clarinet',
 'clock',
 'cloud',
 'coffee cup',
 'compass',
 'computer',
 'cookie',
 'cooler',
 'couch',
 'cow',
 'crab',
 'crayon',
 'crocodile',
 'crow